# Scattering from radial potentials

Here we will use some C++ code to calculate trajectories of particles scattering off of radially symmetric potentials centered at the origin. Using this in bare python is **extremely slow** because there are a lot of iterative loops. Doing this in C++ with the "-O3" compiler setting will allow the compiler to vectorize when possible.

To do that, we will use swig to compute the trajectory. It will output the trajectory as $(r,\phi)$ pairs in a structure like 

`std::vector< std::pair<double,double> >`

We can then read those as `numpy` arrays to plot them in the $(x,y)$ plane.

### Swig it, compile it, add it to the path

In [ ]:
! swig -c++ -python swig/scattering.i
! python swig/setup.py build_ext --inplace

In [ ]:
import sys
import os
sys.path.append( os.path.abspath("swig") )

In [ ]:
import scattering
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

plt.rcParams["axes.labelsize"] = 16
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16



## Simulation for Hard Sphere Scattering

### Plot the potential

In [ ]:
E = 0.705
b_min = 0.6
b_max = 2.0
n_b = 14
V0 = 1.0
hs = scattering.hard_sphere_potential( V0 )

rvals = np.linspace(0.1,1.5,100)
hsvals = [ hs(r) for r in rvals ]
plt.plot(rvals,hsvals)
plt.xlabel("r")
plt.ylabel("V(r)")
plt.show()

### Simulate the trajectory

In [ ]:

for b in np.linspace(b_min, b_max, n_b):
    xs = scattering.CrossSection_hard_sphere_potential( hs, E, b, 5.0, 10000 )
    deflection = xs.calculate_trajectory()
    traj = np.asarray( xs.get_trajectory() )
    x,y = traj[:,0] * np.cos( traj[:,1] ), traj[:,0] * np.sin(traj[:,1])
    plt.plot(x,y, label="%3.1f"%(b))
    plt.xlabel("x")
    plt.ylabel("y")
    
plt.legend(loc='right')
plt.show()

## Simulation for Lennard-Jones potential

### Plot the potential 

In [ ]:
E = 0.705
b_min = 0.6
b_max = 2.0
n_b = 200
V0 = 1.0
lj = scattering.lennard_jones( V0 )

xmin = 0.5
xmax = 3.0
rvals = np.linspace(xmin, xmax, 1000)
ljvals = [lj(r) for r in rvals]
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.plot(rvals,ljvals)
ax.set_xlabel("r")
ax.set_ylabel("V(r)")
ax.set_xlim(xmin, xmax)
ax.set_ylim(-2, 5)
ax.grid()


### Simulate the trajectory

In [ ]:

for b in np.linspace(b_min, b_max, n_b):
    xs = scattering.CrossSection_lennard_jones( lj, E, b, 5.0, 10000 )
    deflection = xs.calculate_trajectory()
    traj = np.asarray( xs.get_trajectory() )
    x,y = traj[:,0] * np.cos( traj[:,1] ), traj[:,0] * np.sin(traj[:,1])
    plt.plot(x,y, label="%3.1f"%(b))
    plt.xlabel("x")
    plt.ylabel("y")
    
#plt.legend(loc='right')
plt.show()

In [ ]:
# Exercise: repeat the above for an inverse-square law, i.e., V(r) = 1/r. 
# Step 1: implement the potential function in scattering.h
# Step 2: compile and run swig. You'll have to restart the kernel to get the new module contents. 
# Step 3: copy-and-paste the above cells, changing the potential to your new inverse square function (functor)